In [4]:
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs


In [5]:
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

sm_sess = sagemaker.session.Session()

AWS Region: eu-west-3
RoleArn: arn:aws:iam::260598086981:role/service-role/AmazonSageMaker-ExecutionRole-20210916T222927


In [6]:
bucket = sagemaker.Session().default_bucket()
bucket

'sagemaker-eu-west-3-260598086981'

In [13]:
prefix = "demo-sagemaker-boston"

In [14]:
s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'knn_model')
s3_output_location

's3://sagemaker-eu-west-3-260598086981/demo-sagemaker-boston/knn_model'

## Entrenamiento del modelo k-nn
- https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/k-nearest-neighbors.html

In [16]:
container=sagemaker.image_uris.retrieve("knn", region)
print(container)

749696950732.dkr.ecr.eu-west-3.amazonaws.com/knn:1


---

### Formato de entrada
- https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/kNN-in-formats.html

In [30]:
train_input = TrainingInput(
    f"s3://{bucket}/{prefix}/data/train.csv", 
    content_type="text/csv",
)
validation_input = TrainingInput(
    f"s3://{bucket}/{prefix}/data/validation.csv",
    content_type="text/csv",
)

In [31]:
knn = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=s3_output_location,
    sagemaker_session=sm_sess,
)

Hyperparametros: https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/kNN_hyperparameters.html

In [33]:
knn.set_hyperparameters(
    feature_dim= 13,
    k = 3, 
    sample_size = 303, 
    predictor_type = 'regressor'
)

___

In [34]:
knn.fit({"train": train_input, "validation": validation_input}, wait=True)

2021-10-21 16:26:24 Starting - Starting the training job...
2021-10-21 16:26:34 Starting - Launching requested ML instancesProfilerReport-1634833584: InProgress
...
2021-10-21 16:27:22 Starting - Preparing the instances for training.........
2021-10-21 16:28:42 Downloading - Downloading input data
2021-10-21 16:28:42 Training - Downloading the training image............
2021-10-21 16:30:50 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/21/2021 16:30:50 INFO 140690016610112] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'_kvstore': 'dist_async', '_log_level': 'info', '_num_gpus': 'auto', '_num_kv_servers': '1', '_tuning_objective_metric': '', '_faiss_index_nprobe': '5', 'epochs': '1', 'feature_dim': 'auto', 'faiss_index_ivf_nlists': 'auto', 'index_metric': 'L2', 'index_type': 'faiss.Flat', 'mini_ba

In [35]:
knn.output_path

's3://sagemaker-eu-west-3-260598086981/demo-sagemaker-boston/knn_model'

In [36]:
knn.latest_training_job.name

'knn-2021-10-21-16-26-24-454'

In [39]:
rule_output_path = knn.output_path + "/" + knn.latest_training_job.name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive

2021-10-21 16:31:23     322358 demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/rule-output/ProfilerReport-1634833584/profiler-output/profiler-report.html
2021-10-21 16:31:23     168700 demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/rule-output/ProfilerReport-1634833584/profiler-output/profiler-report.ipynb
2021-10-21 16:31:19        191 demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/rule-output/ProfilerReport-1634833584/profiler-output/profiler-reports/BatchSize.json
2021-10-21 16:31:19        199 demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/rule-output/ProfilerReport-1634833584/profiler-output/profiler-reports/CPUBottleneck.json
2021-10-21 16:31:19        126 demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/rule-output/ProfilerReport-1634833584/profiler-output/profiler-reports/Dataloader.json
2021-10-21 16:31:19        127 demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/rule-output/ProfilerReport-1634833584/pr

In [40]:
knn.model_data

's3://sagemaker-eu-west-3-260598086981/demo-sagemaker-boston/knn_model/knn-2021-10-21-16-26-24-454/output/model.tar.gz'

___
# Deploy

In [66]:
from sagemaker.predictor import csv_serializer, json_deserializer

In [48]:
knn_predictor=knn.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    #serializer=CSVSerializer()
)
knn_predictor.serializer = csv_serializer
knn_predictor.deserializer = json_deserializer

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


TypeError: __init__() got an unexpected keyword argument 'content_type'

## Test result

In [81]:
import pandas as pd
import numpy as np

In [102]:
data = pd.read_csv(f"s3://{bucket}/{prefix}/data/validation.csv", header=None)

In [105]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,8.8,20.08490,0.0,18.10,0.0,0.700,4.368,91.2,1.4395,24.0,666.0,20.2,285.83,30.63
1,20.8,12.04820,0.0,18.10,0.0,0.614,5.648,87.6,1.9512,24.0,666.0,20.2,291.55,14.10
2,17.4,0.13554,12.5,6.07,0.0,0.409,5.594,36.8,6.4980,4.0,345.0,18.9,396.90,13.09
3,11.9,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88
4,29.8,0.12579,45.0,3.44,0.0,0.437,6.556,29.1,4.5667,5.0,398.0,15.2,382.84,4.56


In [103]:
y = data.iloc[:, 0].values
x = data.iloc[:, 1:].values

In [104]:
print(y.shape, x.shape)

(101,) (101, 13)


In [107]:
result = knn_predictor.predict(x)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [108]:
# Convert JSON predictions to an array
preds_array = np.array([result['predictions'][i]['predicted_label'] for i in range(len(result['predictions']))])

In [119]:
np.power(y - preds_array, 2).mean()

35.27848193687974

### Delete deployed model

In [ ]:
boto3.client("sagemaker").delete_endpoint(EndpointName=predictor.endpoint_name)